In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import selenium.webdriver.support.ui as ui
from pdf2image import convert_from_path
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import requests
import random
import shutil
import glob
import json
import fire
import time
import csv
import sys
import os
import re

pytesseract.pytesseract.tesseract_cmd = r'E:\Tesseract-OCR\tesseract.exe'
base_dir = os.path.abspath(os.path.dirname("__file__"))
os.environ["LANG"] = "en_US.UTF-8"

# 192.41.71.199:3128   50.246.120.125:8080  157.230.44.213:8080   74.121.98.90:8080   192.41.71.221:3128 
# 74.121.98.90:8080  192.144.215.244:3128
PROXY =     "192.41.71.199:3128"   

options = webdriver.ChromeOptions()
appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}
options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False 
# options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
# options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"

driver = webdriver.Chrome(executable_path = r'D:\Chromedriver\chromedriver.exe' ,options=options)
driver.maximize_window()
driver.get("https://www.gfxtra31.com/programming-books/")


# GET SESSION ID & URL
executor_url = driver.command_executor._url
session_id = driver.session_id
print(session_id)
print(executor_url)


be011be5376951264261eeaeebd8e947
http://127.0.0.1:50478


In [ ]:
FPN      = []
FP_URL   = []

for l in range(76,95):
    for i in range(24):
        # To parse all 94 pages
        driver.get('https://www.gfxtra31.com/ebooks/programming-books/page/'+str(l)+'/')
        PN = driver.execute_script('return document.getElementsByClassName("baslik")['+str(i)+'].innerText').split('\n')
        P_URL = driver.execute_script('return document.getElementsByClassName("baslik")['+str(i)+'].innerHTML')
        FPN.append(PN[0])
        FP_URL.append(P_URL.split('href=')[1].split('>')[0].replace('"',''))

data = pd.DataFrame({'Product_Name':FPN,'Product_URL':FP_URL})
data.to_csv('demo.csv',index=False,encoding='UTF-8')

In [ ]:
# To read the collected data from CSV
data = pd.read_csv('demo.csv')

In [ ]:
# File Hoster URL
FH_URL = []
for i in range(13,len(data['Product_URL'])):
    # To send URL to Headless Browser
    driver.get(data['Product_URL'][i])         
    time.sleep(1)     
    """ COLLECT SOUP """
    content = driver.page_source
    soup = BeautifulSoup(content, "html.parser")

    demo = str(soup.findAll('div',attrs={'style':'display:inline;'})[0]).split('display:inline;')[0].split('div id')[1].split('style')[0].replace(' ','')
    try:
        try:
            driver.find_element_by_xpath('//*[@id'+str(demo)+']/p[2]/a').click()
        except Exception as e:
            driver.find_element_by_xpath('//*[@id'+str(demo)+']/p[3]/strong/a').click()
    except Exception as e:
        try:
            driver.find_element_by_xpath('//*[@id'+str(demo)+']/div/a').click()
        except Exception as e:
            driver.find_element_by_xpath('//*[@id'+str(demo)+']/p/a').click()
    # To get current URL of opened tab
    FH_URL.append(driver.current_url)
    print(driver.current_url)
    k += 1
    

In [ ]:
# To make a new DataFrame and make a new CSV file
data2 = data.assign(FH_URLs = FH_URL) 
data2.to_csv('final_data.csv',index = False,encoding='UTF-8')

In [ ]:
# PN     = []      # Product Name
# P_URL  = []      # Product URL
# FH_URL = []      # File Hoster URL

# for l in range(1,95):
#     for k in range(1,25):
#         # To parse all 94 pages 
#         driver.get('https://www.gfxtra31.com/ebooks/programming-books/page/'+str(l)+'/')
#         time.sleep(3)
#         # Product Name 
#         PN.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[1]/div[1]/div[1]/div/div['+str(k)+']/div/div[1]/div[1]/a').text)
#         # Product URL
#         P_URL.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[1]/div[1]/div[1]/div/div['+str(k)+']/div/div[1]/div[1]/a').get_attribute('href'))
#         break
#     break


In [ ]:
# for row in soup.findAll('a',attrs={'rel':'nofollow'}):
#     print(row.get_text)
#     print(row.attrs['href'])
#     print(row.cdata_list_attributes)
#     print(row.children)
#     print(row.extract)